In [ ]:
import cv2
import numpy as np

# Fonction de détection
# `model` est le modèle de détection d'objets chargé local ou non
def detect_objects(image, model, conf_threshold=0.5):
    """
    Détecte les objets dans l'image à l'aide d'un modèle de détection d'objets.
    
    Args:
    - image: L'image d'entrée
    - model: Le modèle de détection (par ex. YOLOv4)
    - conf_threshold: Seuil de confiance pour filtrer les prédictions

    Returns:
    - boxes: Liste des boîtes de délimitation
    - class_ids: Liste des IDs des classes détectées
    - confidences: Liste des scores de confiance
    """
    # Obtenir la largeur et la hauteur de l'image
    h, w = image.shape[:2]
    
    # Faire une prédiction avec le modèle
    results = model(image)
    
    boxes = []
    class_ids = []
    confidences = []
    
    # Parcourir les résultats des prédictions
    for detection in results:
        class_id = detection['class_id']
        confidence = detection['confidence']
        box = detection['bbox']  # Coordonnées de la boîte de délimitation [x, y, largeur, hauteur]
        
        # Filtrer par seuil de confiance
        if confidence >= conf_threshold:
            x, y, width, height = box
            boxes.append([int(x * w), int(y * h), int(width * w), int(height * h)])
            class_ids.append(class_id)
            confidences.append(float(confidence))
    
    return boxes, class_ids, confidences

# Liste des noms de classes
CLASSES = ["rhinocéros", "éléphant"]

def count_classes(class_ids):
    """
    Compte le nombre d'occurrences de chaque classe détectée.
    
    Args:
    - class_ids: Liste des IDs de classes détectées
    
    Returns:
    - class_counts: Dictionnaire avec le nombre d'occurrences de chaque classe
    """
    class_counts = {class_name: 0 for class_name in CLASSES}
    
    for class_id in class_ids:
        class_name = CLASSES[class_id]
        class_counts[class_name] += 1
    
    return class_counts

# Exemple d'utilisation
if __name__ == "__main__":
    # Charge ton modèle ici (par exemple YOLOv4)
    model = load_yolo_model()  # Fonction à implémenter selon ton framework (PyTorch, TensorFlow, etc.)

    # Charge une image pour la détection
    image = cv2.imread("test_image.jpg")

    # Effectuer la détection
    boxes, class_ids, confidences = detect_objects(image, model)

    # Affiche les résultats de comptage
    class_counts = count_classes(class_ids)
    print(f"Nombre de rhinocéros détectés: {class_counts['rhinocéros']}")
    print(f"Nombre d'éléphants détectés: {class_counts['éléphant']}")

    # Affichage des boîtes de délimitation sur l'image (facultatif)
    for i, box in enumerate(boxes):
        x, y, w, h = box
        label = f"{CLASSES[class_ids[i]]}: {confidences[i]:.2f}"
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Enregistrer ou afficher l'image avec les boîtes
    cv2.imwrite("output_image.jpg", image)
    cv2.imshow("Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
